In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cellbayesassay as cba
import pickle
import re
import attila_utils

In [2]:
data_CO28151 = cba.get_data('../../resources/cell-based-assays/raw-data-cleaned-by-tina-CO28151-draft1.xlsx', 'Data')
data_CO28151

,Study,Experiment,Assay,Batch,Plate,TI,Name,conc,concentration,conc_log10,Activity
0,CO28151,Aβ release (H4 cells),Aβ38,1,1,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,4010.00000
1,CO28151,Aβ release (H4 cells),Aβ38,1,1,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,6280.00000
2,CO28151,Aβ release (H4 cells),Aβ38,1,1,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,27790.00000
3,CO28151,Aβ release (H4 cells),Aβ38,1,1,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,32560.00000
4,CO28151,Aβ release (H4 cells),Aβ38,1,1,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,20670.00000
...,...,...,...,...,...,...,...,...,...,...,...
1633,CO28151,Aβ clearance (BV2 cells),Aβ42 SN,2,2,VC,,NaN,1.000000e-09,-9.0,53839.31290
1634,CO28151,Aβ clearance (BV2 cells),Aβ42 SN,2,2,VC,,NaN,1.000000e-09,-9.0,58911.37592
1635,CO28151,Aβ clearance (BV2 cells),Aβ42 SN,2,2,VC,,NaN,1.000000e-09,-9.0,50964.09701
1636,CO28151,Aβ clearance (BV2 cells),Aβ42 SN,2,2,VC,,NaN,1.000000e-09,-9.0,NaN


In [3]:
my_treatments = ['TI21', 'TI22', 'TI26']
treatment2drugname = dict(zip(my_treatments, ['TUDCA', 'Cysteamine', 'Arundine']))
#treatment2drugname = dict(zip(my_treatments, ['Chenodiol', 'Cysteamine', 'Arundine']))

concentrations = {
    ('TI21', 1): 100e-6,
    ('TI21', 2): 10e-6,
    ('TI21', 3): 1e-6,
    ('TI22', 1): 10e-6,
    ('TI22', 2): 3e-6,
    ('TI22', 3): 1e-6,
    ('TI26', 1): 10e-6,
    ('TI26', 2): 3e-6,
    ('TI26', 3): 1e-6,
}



def get_concentration(TI, conc, concentrations=concentrations):
    TI = TI.strip()
    if re.match('^TI.*', TI):
        val = concentrations[(TI, conc)]
    else:
        val = concentrations[TI]
    return(val)


def read_data(sheet_name, assays, mean_sem_col='Group', skiprows=0):
    data = pd.read_excel(fpath, sheet_name=sheet_name, skiprows=skiprows)
    data = data.loc[~ data[mean_sem_col].isin(['mean', 'SEM'])]
    data['TI'] = data.TI.apply(lambda s: re.sub('TI\\s+', 'TI', s)).str.strip()
    data = data.applymap(lambda x: 0 if x == '<LOD' else x)
    data = data.applymap(lambda x: pd.NA if x == 'N/A' else x)
    data_nonassays = data.loc[:, ~ data.columns.isin(assays)]
    data_assays = pd.concat([data[a].astype(np.float64).to_frame(a) for a in assays], axis=1)
    data = pd.concat([data_nonassays, data_assays], axis=1)
    return(data)

def add_concentrations(data, concentrations):
    data['concentration'] = data.apply(lambda r: get_concentration(r.loc['TI'], r.loc['conc'], concentrations=concentrations), axis=1)
    data['conc_log10'] = data.concentration.apply(np.log10)
    return(data)



In [4]:
fpath = '../../resources/cell-based-assays/ideal-effects.csv'
idealfx = pd.read_csv(fpath, index_col=['experiment', 'assay'], usecols=['experiment', 'assay', 'experiment (nice)', 'assay (nice)'])
fpath = '../../resources/cell-based-assays/TI21_22_26_rawdata_sum-edited.xlsx'
datad = {}
assaysd = {}
inputd = {
    'LPS': ('LPS', 'Group', ['IFN-γ', 'IL-10', 'IL-12p70', 'IL-1β', 'IL-2', 'IL-4', 'IL-5', 'IL-6', 'KC/GRO', 'TNF-α']),
    'Abeta Toxicity': ('LC', 'IRN', ['Viability [% VC]']),
    'Abeta release': ('VC', 'Group', ['Aβ38', 'Aβ40', 'Aβ42']),
    'Abeta clearance': ('Aß', 'Group', ['Aβ42 SN', 'Aβ42 Ly']),
    'GF withdrawal': ('VC', 'IRN', ['PI', 'YOPRO', 'MTT', 'LDH']),
    'Tau phosphorylation': ('VC', 'Group', ['Tau [pg/µg prot]', 'pTau (T231)', 'ratio']),
    'NOG': ('VC', 'IRN', ['total neurite length', 'Number of branch points', 'Neurogenesis', 'average neurite length']),
}

for sheet_name, v in inputd.items():
    control, mean_sem_col, assaysl = v
    concentrationsd = concentrations.copy()
    concentrationsd.update({control: 1e-9})
    assaysd.update({sheet_name: assaysl})
    skiprows = 0
    if sheet_name in ['Abeta release', 'Abeta clearance']:
        skiprows = 1
    data = read_data(sheet_name=sheet_name, assays=assaysl, mean_sem_col=mean_sem_col, skiprows=skiprows)
    if sheet_name == 'Abeta release':
        data['TI'] = data.TI.apply(lambda s: re.sub('^VC.*', 'VC', s))
    if sheet_name == 'Abeta clearance':
        data['conc'] = data.conc.str.replace('c', '').astype(np.float64)
    data = add_concentrations(data, concentrations=concentrationsd)
    s = idealfx.xs(sheet_name, 0, 0).loc[assaysl, 'assay (nice)']
    data = data.rename(s.to_dict(), axis=1)
    mycols = data.drop(s.to_list(), axis=1).columns.to_list()
    data['idx'] = data.index
    data['Study'] = 'CO25360'
    data['Experiment'] = idealfx.xs(sheet_name)['experiment (nice)'].iloc[0]
    data = pd.melt(data, id_vars=['idx', 'Study', 'Experiment'] + mycols, value_vars=s.to_list(), var_name='Assay', value_name='Activity')
    data = data.rename({'plate': 'Plate', 'Cohort': 'Batch'}, axis=1)
    datad[sheet_name] = data

data_CO25360 = pd.concat(datad.values()).drop(['idx', 'IRN', 'ID'], axis=1)
data = pd.concat([data_CO28151, data_CO25360], axis=0)
data

/var/folders/49/kvzc40q90ygdgsxzf49p1wjmj6bgxh/T/ipykernel_78882/3208215962.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: 0 if x == '<LOD' else x)
/var/folders/49/kvzc40q90ygdgsxzf49p1wjmj6bgxh/T/ipykernel_78882/3208215962.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if x == 'N/A' else x)
/var/folders/49/kvzc40q90ygdgsxzf49p1wjmj6bgxh/T/ipykernel_78882/3208215962.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: 0 if x == '<LOD' else x)
/var/folders/49/kvzc40q90ygdgsxzf49p1wjmj6bgxh/T/ipykernel_78882/3208215962.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: pd.NA if x == 'N/A' else x)
/var/folders/49/kvzc40q90ygdgsxzf49p1wjmj6bgxh/T/ipykernel_78882/3208215962.py:32: FutureWarning: Data

,Study,Experiment,Assay,Batch,Plate,TI,Name,conc,concentration,conc_log10,Activity,Group
0,CO28151,Aβ release (H4 cells),Aβ38,1.0,1.0,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,4010.0,NaN
1,CO28151,Aβ release (H4 cells),Aβ38,1.0,1.0,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,6280.0,NaN
2,CO28151,Aβ release (H4 cells),Aβ38,1.0,1.0,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,27790.0,NaN
3,CO28151,Aβ release (H4 cells),Aβ38,1.0,1.0,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,32560.0,NaN
4,CO28151,Aβ release (H4 cells),Aβ38,1.0,1.0,TI2,Phenamil methanesulfonate salt,c1,1.000000e-04,-4.0,20670.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
259,CO25360,Neurite outgrowth (primary neurons),longest neurite,NaN,NaN,VC,NaN,NaN,1.000000e-09,-9.0,37.8,NaN
260,CO25360,Neurite outgrowth (primary neurons),longest neurite,NaN,NaN,VC,NaN,NaN,1.000000e-09,-9.0,35.0,NaN
261,CO25360,Neurite outgrowth (primary neurons),longest neurite,NaN,NaN,VC,NaN,NaN,1.000000e-09,-9.0,32.7,NaN
262,CO25360,Neurite outgrowth (primary neurons),longest neurite,NaN,NaN,VC,NaN,NaN,1.000000e-09,-9.0,33.4,NaN


In [5]:
P_CO25360 = pd.read_csv('../../results/2023-09-26-cell-bayes-assays/H102_posteriors.csv', index_col=[0, 1], header=[0, 1])
P_CO25360 = P_CO25360.rename({'TUDCA': 'TI21', 'Cysteamine': 'TI22', 'Arundine': 'TI26'}, axis=1, level=0)
P_CO25360['study'] = 'CO25360'
P_CO25360 = P_CO25360.set_index('study', append=True)
P_CO25360 = P_CO25360.swaplevel(1,2).swaplevel(0, 1).stack(level=0)
P_CO25360

H1  \
study                                                                       
CO25360 Aβ clearance (BV2 cells)                    Aβ42 Ly TI21  0.99275   
                                                            TI22  0.00000   
                                                            TI26  0.87900   
                                                    Aβ42 SN TI21  0.09825   
                                                            TI22  0.00200   
...                                                                   ...   
        Trophic factor withdrawal (primary neurons) PI      TI22  0.00075   
                                                            TI26  0.05100   
                                                    YOPRO   TI21  0.00875   
                                                            TI22  0.00000   
                                                            TI26  0.00050   

                                                                       H0  \
study                                                                       
CO25360 Aβ clearance (BV2 cells)                    Aβ42 Ly TI21  0.00725   
                                                            TI22  0.00000   
                                                            TI26  0.12000   
                                                    Aβ42 SN TI21  0.86375   
                                                            TI22  0.18175   
...                                                                   ...   
        Trophic factor withdrawal (primary neurons) PI      TI22  0.98275   
                                                            TI26  0.94900   
                                                    YOPRO   TI21  0.97275   
                                                            TI22  0.95175   
                                                            TI26  0.97700   

                                                                       H2  
study                                                                      
CO25360 Aβ clearance (BV2 cells)                    Aβ42 Ly TI21  0.00000  
                                                            TI22  1.00000  
                                                            TI26  0.00100  
                                                    Aβ42 SN TI21  0.03800  
                                                            TI22  0.81625  
...                                                                   ...  
        Trophic factor withdrawal (primary neurons) PI      TI22  0.01650  
                                                            TI26  0.00000  
                                                    YOPRO   TI21  0.01850  
                                                            TI22  0.04825  
                                                            TI26  0.02250  

[91 rows x 3 columns]

In [8]:
P_CO28151 = pd.read_csv('../../results/2024-02-14-cell-bayes/H102_posteriors.csv', index_col=[0, 1, 2, 3]).drop('name', axis=1)
P_CO28151

H1       H0       H2
study   experiment               assay   TI                             
CO28151 Aβ clearance (BV2 cells) Aβ42 Ly TI1   0.07300  0.77600  0.15100
                                         TI2   0.99925  0.00075  0.00000
                                         TI3   0.25525  0.69925  0.04550
                                         TI4   0.61225  0.38000  0.00775
                                         TI5   0.99875  0.00125  0.00000
...                                                ...      ...      ...
        Aβ release (H4 cells)    Aβ42    TI11  0.00000  0.37325  0.62675
                                         TI12  0.00000  0.00200  0.99800
                                         TI13  0.02650  0.95625  0.01725
                                         TI14  0.00025  0.95300  0.04675
                                         TI15  0.06325  0.93675  0.00000

[75 rows x 3 columns]

In [ ]:
P = pd.concat([P_CO28151, P_CO28151

In [7]:
%connect_info

{
  "shell_port": 54259,
  "iopub_port": 54260,
  "stdin_port": 54261,
  "control_port": 54263,
  "hb_port": 54262,
  "ip": "127.0.0.1",
  "key": "327b58e3-c6aa0d82d4c6a38d594cf962",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3",
  "jupyter_session": "/Users/jonesa7/CTNS/notebooks/2024-03-04-cell-bayes-vs-frequentist/2024-03-04-cell-bayes-vs-frequentist.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-4e353a2a-1294-4a2b-a833-0b012e3b74e1.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
